
##### Syed Ali Alim Rizvi

# Logistic Regression vs. Bayesian Classifier

In [3]:
library(reshape2)
library(ggplot2)
library(tidyr)
library(mvtnorm) # generates multivariate Gaussian sampels and calculate the densities


ERROR: Error in library(mvtnorm): there is no package called 'mvtnorm'


#### Data Import

In [1]:
t1e.train <- read.csv('LogisticVsBaysian_train.csv')
t1e.test <- read.csv('LogisticVsBaysian_test.csv')

# Logistic

## Steps to Build a Logistic Regression
Taking the following steps is neccesseary to build a logistic regression:
<ol>
	<li>Implement sigmoid function $\sigma(\pmb{w}.\mathbf{x})$, and initialize weight vector $\pmb{w}$, learning rate $\eta$ and stopping criterion $\epsilon$.</li>
	<li>Repeat the followings until the improvement becomes negligible (i.e., $|\mathcal{L}(\pmb{w}^{(\tau+1)})-\mathcal{L}(\pmb{w}^{(\tau)})| \lt \epsilon$):
<ol>
	<li>Shuffle the training data</li>
	<li>For each datapoint in the training data do:
<ol>
	<li>$\pmb{w}^{(\tau+1)} := \pmb{w}^{(\tau)} - \eta (\sigma(\pmb{w}.\mathbf{x}) - t_n) \pmb{x}_n$</li>
</ol>
</li>
</ol>
</li>
</ol>

In the followings, we implement each of these steps.

## Building Linear Regression
Similar to the previous activities, we first define some auxilary functions and then develop the logsitic regresion.
### Auxilary Functions

In [12]:
# auxiliary function that predicts class labels
predict <- function(w, X, c0, c1){
    sig <- sigmoid(w, X)
    return(ifelse(sig>0.5, c1,c0))
}
    
# auxiliary function that calculate a cost function
cost <- function (w, X, T, c0){
    sig <- sigmoid(w, X)
    return(sum(ifelse(T==c0, 1-sig, sig)))
}

**Step 1 (Sigmoid):** Let's define our sigmoid function, first.

In [13]:
# Sigmoid function (=p(C1|X))
sigmoid <- function(w, x){
    return(1.0/(1.0+exp(-w%*%t(cbind(1,x)))))    
}

**Step 1 (initializations):** Now, we initiate the weight vector, learning rate, stopping threshold, etc.

In [14]:
# Initializations
tau.max <- 1000 # maximum number of iterations
eta <- 0.01 # learning rate
epsilon <- 0.01 # a threshold on the cost (to terminate the process)
tau <- 1 # iteration counter
terminate <- FALSE

## Just a few name/type conversion to make the rest of the code easy to follow
X <- as.matrix(train.data) # rename just for conviniance
T <- ifelse(train.label==c0,0,1) # rename just for conviniance

W <- matrix(,nrow=tau.max, ncol=(ncol(X)+1)) # to be used to store the estimated coefficients
W[1,] <- runif(ncol(W)) # initial weight (any better idea?)

# project data using the sigmoid function (just for convenient)
Y <- sigmoid(W[1,],X)

costs <- data.frame('tau'=1:tau.max)  # to be used to trace the cost in each iteration
costs[1, 'cost'] <- cost(W[1,],X,T, c0)

**Step 2:** Here, we use SGD to learn the weight vector. Note that there are two loops. In the outter loop, we shuffle the samples and then start the inner loop. In the inner loop, we visit the training samples one by one and update the weights accordingly.

In [15]:
while(!terminate){
    # check termination criteria:
    terminate <- tau >= tau.max | cost(W[tau,],X,T, c0)<=epsilon
    
    # shuffle data:
    train.index <- sample(1:train.len, train.len, replace = FALSE)
    X <- X[train.index,]
    T <- T[train.index]
    
    # for each datapoint:
    for (i in 1:train.len){
        # check termination criteria:
        if (tau >= tau.max | cost(W[tau,],X,T, c0) <=epsilon) {terminate<-TRUE;break}
        
        Y <- sigmoid(W[tau,],X)
            
        # Update the weights
        W[(tau+1),] <- W[tau,] - eta * (Y[i]-T[i]) * cbind(1, t(X[i,]))
        
        # record the cost:
        costs[(tau+1), 'cost'] <- cost(W[tau,],X,T, c0)
        
        # update the counter:
        tau <- tau + 1
        
        # decrease learning rate:
        eta = eta * 0.999
    }
}
# Done!
costs <- costs[1:tau, ] # remove the NaN tail of the vector (in case of early stopping)

# the  final result is:
w <- W[tau,]
cat('\nThe  final coefficents are:',w)


The  final coefficents are: 0.02115738 -0.9925501 1.041473

## Bayesian Classifier 
### Steps to Build A Bayesian Classifier
These are the steps to build a bayesian Classifier:
<ol>
	<li>Calculate the class priors $p(\mathcal{C}_k)$ based on the relative number of training data in each class,</li>
	<li>Calculate the class means $\mu_k$, class covariance matrices $\mathbf{S}_k$ and shared covariance matrix $\Sigma$ using the training data,</li>
	<li>Using the estimated PDF function, calculate $p(x_n|\mathcal{C}_k)$ for each data point and each class,</li>
	<li>For each test sample, find the class label $\mathcal{C}_k$ that maximizes the $p(\mathcal{C}_k)p(x_n|\mathcal{C}_k)$,</li>
</ol>

In the following we take these steps one by one.

### Implementation of Bayesian Classifier
**Step 1:** Let's start with calculating the class probabilities and compare the obtained values with the real class probabilites.

**Note:** we use `.hat` notation after the name of variables to differentiate the estimations from the original (generative parameter) values. 

In [3]:
# Class probabilities:
p0.hat <- sum(train.label==c0)/nrow(train.data) # total number of samples in class 0 divided by the total nmber of training data
p1.hat <- sum(train.label==c1)/nrow(train.data) # or simply 1 - p1.hat

cat(sprintf('\nThe real class probabilities:\t\t%f, %f\nThe estimated class probabilities:\t%f, %f\n', p0, p1, p0.hat, p1.hat))


The real class probabilities:		0.600000, 0.400000
The estimated class probabilities:	0.612000, 0.388000


**Step 2 (means):** Now, we estimate the class means and compare them with the real means.

In [4]:
# Class means:
mu0.hat <- colMeans(train.data[train.label==c0,])
mu1.hat <- colMeans(train.data[train.label==c1,])

cat(sprintf('\nThe real class means:\t\t%f, %f\t%f, %f\t\nThe estimated class means:\t%f, %f\t%f, %f\t\n', 
          mu0[1], mu0[2], mu1[1], mu1[2],
          mu0.hat[1], mu0.hat[2], mu1.hat[1], mu1.hat[2]))


The real class means:		4.500000, 0.500000	1.000000, 4.000000	
The estimated class means:	4.519526, 0.480368	0.962417, 3.992787	


**Step 2 (variances):** Its time to calculate class variance matrices. Based on these matrices, we can easily calculate the shared covariance matrix as it is a weighted average of the class covariance matrices.

In [5]:
# class covariance matrices:
sigma0.hat <- var(train.data[train.label==c0,])
sigma1.hat <- var(train.data[train.label==c1,])

# shared covariance matrix:
sigma.hat <- p0.hat * sigma0.hat + p1.hat * sigma1.hat 

cat(sprintf('\nThe real class covariance matrix:\n\t%f, %f\n\t%f, %f\nThe estimated covariance matrix:\n\t%f, %f\n\t%f, %f\n', 
          sigma[1], sigma[2], sigma[3], sigma[4],
          sigma.hat[1], sigma.hat[2], sigma.hat[3], sigma.hat[4]))


The real class covariance matrix:
	1.000000, 0.000000
	0.000000, 1.000000
The estimated covariance matrix:
	0.934979, 0.043186
	0.043186, 1.069280


**Step 3:**  Now, we calculate the postoriors based on the estimated parameters in the above steps. 

**Note:** We can easily generate some samples using the learnt parameters. That's why Bayesian classifier is known as a generative model. As an optional actvity, try to generate some points, plot them and compare the visualizations with the original samples.

In [6]:
# calculate posteriors:
posterior0 <- p0.hat*dmvnorm(x=train.data, mean=mu0.hat, sigma=sigma.hat)
posterior1 <- p1.hat*dmvnorm(x=train.data, mean=mu1.hat, sigma=sigma.hat)

**Step 4:** To predict the class label of a data pint, we only need to compare the posteriors and assign the label which is associated with te largest posterior.

In [7]:
# calculate predictions:
train.predict <- ifelse(posterior0 > posterior1, c0, c1)
test.predict <- ifelse(p0.hat*dmvnorm(x=test.data, mean=mu0.hat, sigma=sigma.hat) > p1.hat*dmvnorm(x=test.data, mean=mu1.hat, sigma=sigma.hat), c0, c1)

# calculate accuracy:
cat(sprintf('\nTraining accuracy:\t%.2f%%', sum(train.label==train.predict)/nrow(train.data)*100))
cat(sprintf('\nTesting accuracy:\t%.2f%%', sum(test.label==test.predict)/nrow(test.data)*100))


Training accuracy:	99.80%
Testing accuracy:	99.40%

To see the performance of our classifier, let's produce two confusion matrices (one for training and the other one for testing data).

In [1]:
# Confusion Matix (Train):
table(train.label, train.predict)

# Confusion Matix (Test):
table(test.label, test.predict)

ERROR: Error in table(train.label, train.predict): object 'train.label' not found
